# MODEL-PROJECT: The Black-Scholes(-Merton)-Model

*<span style="color:red">**Before we start please install these packages in your VScode terminal (if not already installed)**</span>*

To do that just remove the '#' in the codecell below and run the code once. Be sure to put the '#' back in before you continue as to not run into problems when rerunning the code.

In [1]:
# !pip install yfinance
# !pip install ipywidgets
# !pip install dash
# !pip install dash-core-components

Imports and set magics:

In [2]:
import datetime
import numpy as np
import sympy as sp
import yfinance as yf
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import norm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# Model description

The Black-Scholes model is a mathematical model for pricing European-style options. It's widely used in the financial industry, and it has significantly impacted the field of financial economics. The model assumes that the price of the underlying asset follows a geometric Brownian motion, and it derives a closed-form solution for the price of said asset. It provides a theoretical estimate of the value of a call or put option based on the following parameters:

-The current price of the underlying asset (S)  
-The strike price of the option (K)  
-The time to maturity (T)  
-The risk-free interest rate (r)  
-The volatility of the underlying asset (σ)  

Deriving the Black-Scholes PDE for a call-option on a non-dividend paying stock with strike K and maturity T. We assume that the stock price follows a geometric Brownian motion so that:

$ d (S_t) = μS_t dt + σS_tdW_t $ (i)

where $ W_t $ is a standard Brownian motion. We also assume that interest rates are constant so that 1 unit of currency invested in the cash account at time 0 will be worth $ B_t := exp(rt) $ at time t. We will denote by $ C(S, t) $ the value of the call option at time t. By Itˆo’s lemma we know that:

$ dC(S, t) = μS_t\frac{∂ C}{∂ S} + \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} dt + σS_t\frac{∂ C}{∂ S}dW_t$ (ii)

Let us now consider a self-financing trading strategy where at each time t we hold $x_t$ units of the cash account and $y_t$ units of the stock. Then $P_t$, the time t value of this strategy satisfies:

$P_t = x_tB_t + y_tS_t $ (iii)

We will choose $x_t$ and $y_t$ in such a way that the strategy replicates the value of the option. The self-financing assumption implies that:

$ dP_t = x_tdB_t + y_tdS_t$  (iv)  
$      = rx_tB_tdt + y_t(μS_tdt + σS_tdW_t) $  
$      = (rx_tB_t + y_tμS_t)dt + y_tσS_tdW_t $ (v)

We can equate terms in (ii)) with the corresponding terms in (v) to obtain:

$ y_t = \frac{∂ C}{∂ S} $  (vi)

$ rx_tB_t = \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} $  (vii)

If we set $C_0 = P_0$, the initial value of our self-financing strategy, then it must be the case that $C_t = P_t$ for all t since C and P have the same dynamics. This is true by construction after we equated terms in (ii) with the corresponding terms in (v). Substituting (vi) and (vii) into (iii) we obtain:


$ rS_t\frac{∂ C}{∂ S} + \frac{∂ C}{∂ t} + \frac{1}{2}σ^2S^2\frac{∂^2C}{∂S^2} - rC = 0 $ (viii) 


the Black-Scholes PDE. In order to solve (viii) boundary conditions must also be provided. In the case of the call option, these conditions are as follows:

$ C(S, T) = max(S-K, 0), C(0, t) = 0 $ for all t and $C(S, t) ⭢ S $ $as$ $ S ⭢ ∞ $  

We are finally able to arrive at the final formula for a european style call option as the solution to equation (viii):

$ C(S, K, T, r, σ) = S N(d1) - K e^{-rT} N(d2) $ 

where:

$ d1 = \frac{log(\frac{S}{K}) + (r + \frac{σ^2}{2})T}{σ\sqrt{T}} $

$ d2 = d1 - σ\sqrt{T} $  

and $N(x)$ is the cumulative distribution function of the standard normal distribution.

Equivalenty, from what we call the put-call parity we can also easily compute the price of a European put option using the formula for a call. By doing so we arrive at the following:

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S N(-d1) $  
  
  
In this project, we'll implement the Black-Scholes model using Python and perform various analyses, including numerical solutions for different real world assets and visualizations.

example


## Analytical solution

In our analytical solution below we are using sympy to define the Black and Scholes equation by coding the equations for $d1$ and $d2$ as follows:

In [3]:
# Define the Black-Scholes Model symbols
S, K, T, r, sigma = sp.symbols('S K T r sigma')

# Define the Black-Scholes Model equation
d1 = ((r + 0.5*sigma**2)*T + sp.log(S/K)) / (sigma*sp.sqrt(T))
d1

(T*(r + 0.5*sigma**2) + log(S/K))/(sqrt(T)*sigma)

In [4]:
d2 = d1 - sigma*sp.sqrt(T)
d2

-sqrt(T)*sigma + (T*(r + 0.5*sigma**2) + log(S/K))/(sqrt(T)*sigma)

Once this has been done, we are using the lambdify function from sympy to convert the sympy equations into lambda functions that we can then use in our numerical solution. Additionally, once this has been done, we create an if statement that will implement either the call or put equations when finding the price of our options:

In [5]:
# Define lambdified functions for d1 and d2
d1_lambdified = sp.lambdify((S, K, T, r, sigma), d1, 'numpy')
d2_lambdified = sp.lambdify((S, K, T, r, sigma), d2, 'numpy')

# Define the Black-Scholes Model equation
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = d1_lambdified(S, K, T, r, sigma)
    d2 = d2_lambdified(S, K, T, r, sigma)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
    return price

## Numerical solution

What we do below is define initial values for our independent variables and look for numerical solutions (option prices) of the Black and Scholes equation for a range of strike prices (K) for both call and put options. In doing so, we are able to plot the results of our numerical analysis and create an interactive plot that allows us to adjust our variable values as we see fit to illustrate how increasing/decreasing the select variables causes the options prices to change in response.

In [6]:
# Define the initial values for K, S, T, r, and sigma
K = 100
S = 100
T = 1
r = 0.05
sigma = 0.2

# Define a range of strike prices
stock_prices = np.linspace(1, 200, 100)

# Create a function to update the plot with the new value of sigma
def update_plot(K, T, r, sigma):
    call_prices1 = [black_scholes(S, K, T, r, sigma, option_type='call') for S in stock_prices]
    put_prices1 = [black_scholes(S, K, T, r, sigma, option_type='put') for S in stock_prices]
    
    plt.figure(figsize=(10, 6))
    plt.plot(stock_prices, call_prices1, label="Call prices")
    plt.plot(stock_prices, put_prices1, label="Put prices")
    plt.title(f"Black-Scholes Model for K={K}, T={T}, r={r}, sigma={sigma}")
    plt.xlabel("Stock Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.show()

# Create a slider widget for S
K_slider = widgets.FloatSlider(
    value=K,
    min=S * 0.5,
    max=S * 1.5,
    step=1,
    description='Strike price "K":',
    continuous_update=False
)    

# Create a slider widget for T
T_slider = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

# Create a slider widget for r
r_slider = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

# Create a slider widget for sigma
sigma_slider = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create an output widget to display the plot
out1 = widgets.Output()

# Display the interactive plot
widgets.interact(update_plot, K=K_slider, T=T_slider, r=r_slider, sigma=sigma_slider)
display(out1)

interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Strike price "K":', max=1…

Output()

Varying the strike price, time to maturity, risk-free rate, and sigma can all affect the option prices according to the Black-Scholes model in the following ways:

- **Strike price (K)**: The strike price is the price at which the option can be exercised. Increasing the strike price of a call option, for example, means the underlying asset must appreciate more to become profitable for the option holder. Thus, increasing the strike price of a call option (holding all else constant) will generally decrease its price, and vice versa for a put option.

- **Time to maturity (T)**: The time to maturity is the amount of time remaining until the option contract expires. All else being equal, an option with a longer time to maturity has a higher chance of becoming profitable, as there is more time for the underlying asset to move in the desired direction. Thus an increase in time to maturity will increase the price of both call and put options.

- **Risk-free rate (r)**: The risk-free rate is the theoretical rate of return of an investment with zero risk. It is the rate at which an investor can earn money with certainty, and is typically tied to the yield on government bonds/treasuries. The risk-free rate is used in the Black-Scholes model to discount the expected future payoff of the option to its present value. An increase in the risk-free rate increases the discount rate, which reduces the present value of the expected future payoff. Thus, an increase in the risk-free rate will decrease the price of both call and put options.

- **Volatility (sigma)**: Volatility measures the degree of variation of a financial asset's price over time. The Black-Scholes model assumes that the volatility of the underlying asset is constant over time. An increase in volatility increases the expected range of future prices for the underlying asset, and thus increases the probability that the option will be exercised profitably. Thus, all else being equal, an increase in volatility will increase the price of both call and put options.

By using the slider to adjust multiple variables at a time, we can get a deeper understanding of how option prices vary when multiple variables increase/decrease simultaneously.


# Example based on real world stock data 

Our analysis below obtains stock data from 15 companies in different industries that we will incorporate into the Black and Scholes model to get a sense for real world data application. We begin by creating a dictionary with the names of the companies and their corresponding stock ticker symbols. Then, we calculate the historical volatility (sigma) for each stock using the Yahoo Finance API and store the information into a seperate dictionary. The function "extract_sigma" is defined and is used to extract the sigma value for a given stock ticker.  
*<span style="color:red">Please note that the average future volatility of the underlying asset (which is our sigma) is the only one parameter in the model that cannot be directly observed in the market. Using historical values of sigma like we did below is therefore a massive oversimplification of how this model is actually applied and is just done for illustration purposes (finding a reliable way to estimate future sigmas is sort of the holy grail and could lead to incredible profit opportunities. One way this is commonly done is by deriving sigma from the prices of other options, whose values are increasing in sigma).</span>*

Our function "plot_black_scholes" takes in a company name and uses the Black-Scholes formula to calculate and plot the call and put prices for a range of strike prices. The strike prices are determined by the stock price and are arbitrarily set between 50% and 150% of the stock price to allow for better visualization that is not heavily skewed. The function uses the stock price, time to maturity (T), risk-free rate (r), and the historical volatility (sigma) to calculate the option prices as done in our numerical solution. 

By adding all of this data into an interactive plot, the user is able to simply choose a value for time to maturity and the risk-free rate and see how the Black and Scholes can be used in a real world case(s). 

In [7]:
# Creating a dictionary of interesting stocks and their respective ticker symbols
company_list = ["Apple", "Microsoft", "Google", "Amazon", "Tesla", "Netflix", "Berkshire Hathaway", 
                "JPMorgan Chase", "Johnson & Johnson", "Visa", "Walmart", "UnitedHealth Group", 
                "Procter & Gamble", "NVIDIA", "Home Depot"] 
ticker_list = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NFLX", "BRK-B", "JPM", "JNJ", "V", "WMT", 
               "UNH", "PG", "NVDA", "HD"]
company_ticker_dict = {company: ticker for company, ticker in zip(company_list, ticker_list)}


# Get today's date and the date exactly one year ago in string format for calculation of historical sigma 
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
yesterday_last_year = today - datetime.timedelta(days=366)
yesterday_str = yesterday.strftime("%Y-%m-%d")
yesterday_one_year_ago_str = yesterday_last_year.strftime('%Y-%m-%d')


# Loop through the list of stock-tickers, calculate the historical volatility (sigma) and store in a dictionary
historical_sigma_dict = {}                                                                      
for ticker in ticker_list:
    data = yf.download(ticker, start=yesterday_one_year_ago_str, end=yesterday_str, progress=False);    # Download price data 
    trading_days = len(data)                                                                            # Determine number of trading days
    returns = data["Close"].pct_change()                                                                # Calculate the daily returns
    std_dev = np.std(returns)                                                                           # Calculate standard deviation of daily returns
    annual_volatility = std_dev * np.sqrt(trading_days)                                                 # Calculate annualized volatility (sigma)
    historical_sigma_dict[ticker] = annual_volatility                                                   # Add results to dictionary


# Defining a function that returns the sigma for a given stock ticker out of the dictionary  
def extract_sigma(stock_ticker):
    return historical_sigma_dict[stock_ticker]


# Creating a Function to plot the Black-Scholes model for a given stock ticker
def plot_black_scholes(company, T_val=T, r_val=r):
    with out2:    
        try:
            # Get stock data through the yfinance module for stocks in the dictionary
            stock_ticker = company_ticker_dict[company]
            stock_data = yf.download(stock_ticker, progress=False)
            stock_price = stock_data.iloc[-1]["Close"]
            stock_sigma = extract_sigma(stock_ticker)

            # Calculate call and put prices for a range of strike prices
            strike_prices2 = np.linspace(stock_price * 0.5, stock_price * 1.5, 100)
            call_prices2 = [black_scholes(stock_price, K, T_val, r_val, stock_sigma, option_type='call') for K in strike_prices2]
            put_prices2 = [black_scholes(stock_price, K, T_val, r_val, stock_sigma, option_type='put') for K in strike_prices2]

            # Clear the output and create the plot
            out2.clear_output(wait=True)
            plt.figure(figsize=(10, 6))
            plt.plot(strike_prices2, call_prices2, label="Call prices")
            plt.plot(strike_prices2, put_prices2, label="Put prices")

            plt.title(f"Black-Scholes Model for {company}")
            plt.xlabel("Strike Price")
            plt.ylabel("Option Price")
            plt.legend()
            plt.show()
        except IndexError as e:
            print(f"Error: {e}. Data for {company} might not be available or the stock might be delisted.")


# Create a dropdown menu for company selection and the sliders for the widgets 
company_dropdown_3 = widgets.Dropdown(
    options=company_list,
    value=company_list[0],
    description="Company:",
)

T_slider_2 = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

r_slider_2 = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)


# Create an Output widget and display the interactive plot
out2 = widgets.Output()
interactive_plot = widgets.interact(plot_black_scholes, company=company_dropdown_3, T_val=T_slider_2, r_val=r_slider_2)
display(out2)

interactive(children=(Dropdown(description='Company:', options=('Apple', 'Microsoft', 'Google', 'Amazon', 'Tes…

Output()

# Extension of the Model - Dividends

In the standard Black & Scholes model we have covered above, dividends are assumed to be zero and are not incorporated into the pricing of options. However, we can extend on this basic model by accomodating for dividends and seeing how this change affects the prices of both call and put options.

In the presence of dividends, the drift rate is adjusted to reflect the present value of future dividends. Specifically, the drift rate becomes:

$ (r - q) $

where $r$ is the risk-free interest rate and $q$ is the continuous dividend yield.

When dividends are included, the Black-Scholes model assumes that the stock price follows a geometric Brownian motion with the drift rate adjusted by the dividend yield. 

With this extension of the model, we can show how the dynamics of the option price satisfy:

$ C(S, K, T, r, σ) = S  e^{-qT} N(d1) - K e^{-rT} N(d2) $   

$ P(S, K, T, r, σ) = K e^{-rT} N(-d2) - S e^{-qT} N(-d1) $  

where:

$ d1 = \frac{log(\frac{S}{K}) + (r-q + \frac{σ^2}{2})T}{σ\sqrt{T}} $  

$ d2 = d1 - σ\sqrt{T} $  

Notice that the dividend yield, q, appears in the formula for $d1$ and in the term $e^{-qT}$, which adjusts the stock price for the present value of the dividends. This term reflects the idea that a dividend payment reduces the value of the stock, as it represents cash flow leaving the company.

Essentially,

Incorporating dividends into the Black-Scholes model allows for a more accurate calculation of option prices, especially for options on stocks that pay significant dividends. This is important for investors who want to make informed decisions about buying or selling options based on their expected value and risk.

In [8]:
# Define the Black-Scholes Model equation
def black_scholes_div(S, K, T, r, q, sigma, option_type='call'):
    d1 = ((r-q + 0.5*sigma**2)*T + np.log(S/K)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-q*T)*norm.cdf(-d1)
    
    return price

# Adding a starting value for the dividend yield "q" oriented on the long term average of the SP500
q = 0.015

# Create a function to update the plot with the new value of sigma
def update_plot_div(K, T, r, q, sigma):
    call_prices3 = [black_scholes_div(S, K, T, r, q, sigma, option_type='call') for S in stock_prices]
    put_prices3 = [black_scholes_div(S, K, T, r, q, sigma, option_type='put') for S in stock_prices]
    
    plt.figure(figsize=(10, 6))
    plt.plot(stock_prices, call_prices3, label="Call prices")
    plt.plot(stock_prices, put_prices3, label="Put prices")
    plt.title(f"Black-Scholes Model for K={K}, T={T}, r={r}, q={q}, sigma={sigma}")
    plt.xlabel("Stock Price")
    plt.ylabel("Option Price")
    plt.legend()
    plt.show()

# Create a slider widget for K
K_slider_3 = widgets.FloatSlider(
    value=K,
    min=S*0.5,
    max=S*1.5,
    step=1,
    description='Strike price "K":',
    continuous_update=False
)    

# Create a slider widget for T
T_slider_3 = widgets.FloatSlider(
    value=T,
    min=1,
    max=10,
    step=1,
    description='Time to maturity "T":',
    continuous_update=False
)

# Create a slider widget for r
r_slider_3 = widgets.FloatSlider(
    value=r,
    min=0,
    max=0.2,
    step=0.005,
    description='Risk free rate "r":',
    continuous_update=False
)

# Create a slider widget for sigma
sigma_slider_3 = widgets.FloatSlider(
    value=sigma,
    min=0.01,
    max=1,
    step=0.01,
    description='Sigma:',
    continuous_update=False
)

# Create a slider widget for q
q_slider_3 = widgets.FloatSlider(
    value=q,
    min=0,
    max=0.15,
    step=0.01,
    description='Dividend Yield:',
    continuous_update=False
)

# Create an output widget to display the plot
out3 = widgets.Output()

# Display the interactive plot
widgets.interact(update_plot_div, K=K_slider_3, T=T_slider_3, r=r_slider_3, sigma=sigma_slider_3, q=q_slider_3)
display(out3)

interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Strike price "K":', max=1…

Output()

When a stock pays a dividend, the price of the stock is expected to drop by an amount equal to the dividend paid. This drop in the stock price reduces the potential profit that can be earned from holding the call option. 

As a result, when the dividend yield of the underlying stock increases, the value of the call option decreases. Conversely, when the dividend yield decreases, the value of the call option increases. 

The drop in the stock price increases the potential profit that can be earned from holding the put option. 

As a result, when the dividend yield of the underlying stock increases, the value of the put option increases. Conversely, when the dividend yield decreases, the value of the put option decreases.

One key result, which can be inferred from simply looking at the formula and the interaction of the variables, is how a longer time to maturity + an increasing dividend yield has a drastic effect on stock price and by extension, the call/put price as well (we can assume that the timeframe is at the very least on a per quarter basis as that is a good representation of dividend payoffs).


# Other possible extensions:

There are several possible more complex extensions to the Black-Scholes model which are briefly described below:

**(i)** Stochastic volatility models: These models assume that the volatility of the underlying asset is not constant but rather follows a stochastic process. The most famous example of this is the Heston model, which is widely used in practice.

**(ii)** Jump-diffusion models: These models allow for sudden, large movements in the underlying asset price, known as jumps which may be as a result of certain macro shocks. They are useful for pricing options on assets with fat-tailed distributions.

**(iii)** Interest rate models: These models take into account the fact that interest rates are not constant but rather follow a stochastic process. One well-known example is the Hull-White model that calculates the price of a derivative security as a function of the entire yield curve rather than a single rate.

**(iv)** American options: The Black-Scholes model assumes that options can only be exercised at expiration. In reality, most options are American-style, which means they can be exercised at any time before expiration. Pricing American options requires the use of numerical methods, such as the binomial tree or Monte Carlo simulation.

**(v)** Path-dependent options: The Black-Scholes model assumes that the payoff of an option depends only on the value of the underlying asset at expiration. Path-dependent options, such as Asian options or barrier options, have payoffs that depend on the path of the underlying asset during the life of the option. Pricing these options requires more sophisticated models and numerical methods.

# Conclusion

he Black-Scholes model is an important model for pricing options, and it has been widely used by traders and investors for decades, but it is not without its drawbacks.

The main takeaways from using the Black-Scholes model in options pricing are as follows:

1. The model provides a theoretical framework for determining the fair price of European options, which are options that can only be exercised at the expiration date.

2. The model assumes that the underlying asset follows a geometric Brownian motion and that the option price is a function of five variables: the current stock price, the strike price, the time to expiration, the risk-free interest rate, and the volatility of the stock price.

3. The Black-Scholes model assumes that the market is efficient and that all relevant information is already incorporated into the stock price. Therefore, it assumes that there are no arbitrage opportunities in the market.

4. The model provides insights into the relationship between option prices and the underlying asset price, time to expiration, and volatility. For example, the price of a call option increases as the stock price increases, the time to expiration increases, or the volatility increases.

5. The model has limitations and assumptions that may not hold in the real world, such as the assumption of constant volatility, the assumption of continuous trading, and the assumption of no transaction costs.

Some of the limitations posed by the model are:

1. Assumptions: The model relies on several assumptions, such as constant volatility, no dividends (base model), efficient markets, and risk-free interest rates. In reality, these assumptions may not always hold true, and this can lead to inaccuracies in the model's predictions.

2. Limited applicability: The model is most suitable for European-style options, which can only be exercised at expiration. It is less effective for American-style options, which can be exercised at any time before expiration.

3. Volatility: While the Black-Scholes model assumes constant volatility, volatility in real markets can vary significantly. This can lead to discrepancies between the model's predictions and actual market prices.

4. Other factors: The model does not take into account other factors that can affect option prices, such as market liquidity, trading volume, and changes in supply and demand.

The base model does not incorporate dividends, but through our extension we are able to work around this limitation and see how options prices change when a company issues dividends. 

All in all, The Black-Scholes model is an elegant model, but it does not perform very well in practice, given the uncertain nature of stock movement, shocks to the macroeconomic enviornment and a variety of other factors that influence stock behaviour and investor sentiment.

